# Dataset Log: 

# In this project, we are using the following datasets:


### 1. Multi-Scale Ultra High Resolution (MUR) Sea Surface Temperature (SST)

In [15]:
# Set up
import s3fs
import xarray as xr
import re

In [6]:
# Bypass AWS tokens, keys etc.
s3 = s3fs.S3FileSystem(anon=True)

# Verify that we're in the right place
sst_files = s3.ls("mur-sst/zarr-v1/")
sst_files

['mur-sst/zarr-v1/',
 'mur-sst/zarr-v1/.zattrs',
 'mur-sst/zarr-v1/.zgroup',
 'mur-sst/zarr-v1/.zmetadata',
 'mur-sst/zarr-v1/analysed_sst',
 'mur-sst/zarr-v1/analysis_error',
 'mur-sst/zarr-v1/lat',
 'mur-sst/zarr-v1/lon',
 'mur-sst/zarr-v1/mask',
 'mur-sst/zarr-v1/sea_ice_fraction',
 'mur-sst/zarr-v1/time']

In [7]:
# Open MUR SST
ds = xr.open_zarr(
        store=s3fs.S3Map(
            root=f"s3://{sst_files[0]}", s3=s3, check=False
        )
)

ds

<xarray.Dataset> Size: 117TB
Dimensions:           (time: 6443, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 52kB 2002-06-01T09:00:00 ... 2020...
Data variables:
    analysed_sst      (time, lat, lon) float64 33TB dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 33TB dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    mask              (time, lat, lon) float32 17TB dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 33TB dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

### 2. NSF NCAR Curated ECMWF Reanalysis 5 (ERA5)

In [8]:
# Verify that we're in the right place
files = s3.ls("nsf-ncar-era5/")
files

['nsf-ncar-era5/e5.oper.an.pl',
 'nsf-ncar-era5/e5.oper.an.sfc',
 'nsf-ncar-era5/e5.oper.an.vinteg',
 'nsf-ncar-era5/e5.oper.fc.sfc.accumu',
 'nsf-ncar-era5/e5.oper.fc.sfc.instan',
 'nsf-ncar-era5/e5.oper.fc.sfc.meanflux',
 'nsf-ncar-era5/e5.oper.fc.sfc.minmax',
 'nsf-ncar-era5/e5.oper.invariant',
 'nsf-ncar-era5/index.html']

In [16]:
# 2) Find NetCDF files for a variable (here: 2m temperature  -  "2t")
# Note: NetCDF files organized by variable, not a single multi-var Zarr.
def find_var_files(prefix: str, var_keys=("2t",), limit=24, maxdepth=5):
    pat = re.compile(r"|".join(map(re.escape, var_keys)), re.I)
    hits = []
    for key in s3.find(prefix, maxdepth=maxdepth):
        if key.endswith(".nc") and pat.search(key):
            hits.append(f"s3://{key}")
            if len(hits) >= limit:
                break
    return hits

In [18]:
paths = find_var_files("nsf-ncar-era5/e5.oper.an.sfc", var_keys=("2t",), limit=12)
paths

['s3://nsf-ncar-era5/e5.oper.an.sfc/194001/e5.oper.an.sfc.128_167_2t.ll025sc.1940010100_1940013123.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194002/e5.oper.an.sfc.128_167_2t.ll025sc.1940020100_1940022923.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194003/e5.oper.an.sfc.128_167_2t.ll025sc.1940030100_1940033123.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194004/e5.oper.an.sfc.128_167_2t.ll025sc.1940040100_1940043023.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194005/e5.oper.an.sfc.128_167_2t.ll025sc.1940050100_1940053123.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194006/e5.oper.an.sfc.128_167_2t.ll025sc.1940060100_1940063023.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194007/e5.oper.an.sfc.128_167_2t.ll025sc.1940070100_1940073123.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194008/e5.oper.an.sfc.128_167_2t.ll025sc.1940080100_1940083123.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194009/e5.oper.an.sfc.128_167_2t.ll025sc.1940090100_1940093023.nc',
 's3://nsf-ncar-era5/e5.oper.an.sfc/194010/e5.oper.an.sfc.128_16

In [20]:
# Open with xarray (NetCDF4 on S3; use h5netcdf or netcdf4 engine)
ds = xr.open_mfdataset(
    paths,
    engine="h5netcdf",                     # good with remote S3 objects
    combine="by_coords",
    chunks={},                             # add dask chunks if you like, e.g. {"time": 24}
    backend_kwargs={"storage_options": {"anon": True}},
)
ds

<xarray.Dataset> Size: 36GB
Dimensions:    (time: 8784, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 70kB 1940-01-01 ... 1940-12-31T23:00:00
Data variables:
    VAR_2T     (time, latitude, longitude) float32 36GB dask.array<chunksize=(27, 139, 277), meta=np.ndarray>
    utc_date   (time) int32 35kB dask.array<chunksize=(744,), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB1 data to netCDF4.
    NETCDF_VERSION:       4.8.1
    CONVERSION_PLATFORM:  Linux r1i7n2 4.12.14-95.51-default #1 SMP Fri Apr 1...
    CONVERSION_DATE:      Thu Mar 16 18:41:53 MDT 2023
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Thu Mar 16 18:42:14 2023: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 5.0.3 (Homepage = http://n...

### 3. Indian Ocean Data: https://ocean-satellite-tools.github.io/mind-the-chl-gap/notebooks/IO_Zarr.html

In [10]:
ds = xr.open_dataset(
    "gcs://nmfs_odp_nwfsc/CB/mind_the_chl_gap/IO.zarr",
    engine="zarr",
    backend_kwargs={"storage_options": {"token": "anon"}},
    consolidated=True
)
ds

<xarray.Dataset> Size: 66GB
Dimensions:                       (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat                           (lat) float32 708B 32.0 31.75 ... -11.75 -12.0
  * lon                           (lon) float32 964B 42.0 42.25 ... 101.8 102.0
  * time                          (time) datetime64[ns] 129kB 1979-01-01 ... ...
Data variables: (12/27)
    CHL                           (time, lat, lon) float32 3GB ...
    CHL_cmes-cloud                (time, lat, lon) uint8 686MB ...
    CHL_cmes-gapfree              (time, lat, lon) float32 3GB ...
    CHL_cmes-land                 (lat, lon) uint8 43kB ...
    CHL_cmes-level3               (time, lat, lon) float32 3GB ...
    CHL_cmes_flags-gapfree        (time, lat, lon) float32 3GB ...
    ...                            ...
    ug_curr                       (time, lat, lon) float32 3GB ...
    v_curr                        (time, lat, lon) float32 3GB ...
    v_wind                        (time, lat, lon) float32 3GB ...
    vg_curr                       (time, lat, lon) float32 3GB ...
    wind_dir                      (time, lat, lon) float32 3GB ...
    wind_speed                    (time, lat, lon) float32 3GB ...
Attributes: (12/92)
    Conventions:                     CF-1.8, ACDD-1.3
    DPM_reference:                   GC-UD-ACRI-PUG
    IODD_reference:                  GC-UD-ACRI-PUG
    acknowledgement:                 The Licensees will ensure that original ...
    citation:                        The Licensees will ensure that original ...
    cmems_product_id:                OCEANCOLOUR_GLO_BGC_L3_MY_009_103
    ...                              ...
    time_coverage_end:               2024-04-18T02:58:23Z
    time_coverage_resolution:        P1D
    time_coverage_start:             2024-04-16T21:12:05Z
    title:                           cmems_obs-oc_glo_bgc-plankton_my_l3-mult...
    westernmost_longitude:           -180.0
    westernmost_valid_longitude:     -180.0